# Getting Started with Echoflow: Processing EK60 Data to Extract Target Strength

Welcome to this beginner-friendly notebook that guides you through the process of using `echoflow` to process EK60 data from the SH1707 survey. `echoflow` is a powerful tool for ocean acoustics data processing. In this notebook, we'll walk through the following steps:

1. **Setting Up**: Importing necessary libraries and setting up the configuration paths for the dataset and pipeline.

2. **Getting Data**: Using `glob_url` to retrieve a list of URLs matching the specified pattern of raw EK60 data files.

3. **Preparing Files**: Extracting file names from URLs and creating a file listing for the transect.

4. **Processing with echoflow**: Starting the echoflow processing using the specified configurations.

5. **Results**: Displaying the first entry from the processed data.

Let's get started!

# Step 0: Install Echoflow

Before we proceed with the instructions on using `echoflow`, let's ensure that we have properly set up and prepared our system with echoflow for usage. Feel free to skip this step if you have echoflow already set up.

### Step 0.1: Make a virtual environment

To keep your Echoflow environment isolated, it's recommended to create a virtual environment using Conda or Python's built-in venv module. Here's an example using Conda:

```bash
conda create --name echoflow-env
conda activate echoflow-env
```

Or, using Python's venv:

```bash
python -m venv echoflow-env
source echoflow-env/bin/activate  # On Windows, use `echoflow-env\Scripts\activate`
```
### Step 0.2: Clone the Repository
Now that you have a virtual environment set up, you can clone the Echoflow project repository to your local machine using the following command:

```bash
git clone <echoflow_repo>
```

### Step 0.3: Install the Package
Navigate to the project directory you've just cloned and install the Echoflow package. The -e flag is crucial as it enables editable mode, which is especially helpful during development and testing. Now, take a moment and let the echoflow do its thing while you enjoy your coffee.

```bash
cd <project_directory>
pip install -e .
```

---

### Step 0.4: Initialize Echoflow and Prefect

To kickstart your journey with Echoflow and Prefect, follow these simple initialization steps:

##### 0.4.1 Initializing Echoflow
Begin by initializing Echoflow with the following command:

```bash
echoflow init
```

This command sets up the groundwork for your Echoflow environment, preparing it for seamless usage.

##### 0.4.2 Initializing Prefect
For Prefect, initialization involves a few extra steps, including secure authentication. Enter the following command to initiate the Prefect authentication process:

- If you have a Prefect Cloud account, provide your Prefect API key to securely link your account. Type your API key when prompted and press Enter.

```bash
prefect cloud login
```

- If you don't have a Prefect Cloud account yet, you can use local prefect account. This is especially useful for those who are just starting out and want to explore Prefect without an account.

```bash
prefect profiles create echoflow-local
```


The initialization process will ensure that both Echoflow and Prefect are properly set up and ready for you to dive into your cloud-based workflows.

---

## Step 1: Setting Up

We begin by importing the required libraries and specifying the paths for the dataset and pipeline configuration files. These files contain the necessary information for data processing.

In [1]:
from pathlib import Path
from echoflow import echoflow_start, glob_url

dataset_config = Path("./datastore.yaml").resolve()
pipeline_config = Path("./pipeline.yaml").resolve()

## Step 1.1
### Pipeline Configuration: Target Strength Processing
In this section, we will provide you with the pipeline configuration that we'll be using for our target strength processing. The configuration is presented in YAML format, which is a structured and human-readable way to define settings for data processing.

Here's the configuration we'll be using:

```yaml
active_recipe: target_strength 
use_local_dask: true
n_workers: 5
pipeline:
- recipe_name: target_strength
  stages:
  - name: echoflow_open_raw
    module: echoflow.stages.subflows.open_raw
    options:
      save_raw_file: true
      use_raw_offline: true
      use_offline: true
  - name: echoflow_compute_TS
    module: echoflow.stages.subflows.compute_TS
    options:
      use_offline: true
```

Let's break down the components of this configuration:

- **active_recipe**: Specifies the recipe to be used for processing, which is set as "target_strength" in this case.

- **use_local_dask**: This flag indicates that we'll be utilizing a local Dask Cluster for parallel processing.

- **n_workers**: Determines the number of worker processes in the Dask Cluster. Here, we're using 5 workers for efficient parallelization.

- **pipeline**: This section defines the sequence of stages to execute. In this example, we're following the "target_strength" recipe, which comprises two stages.

- **echoflow_open_raw**: This stage utilizes the `open_raw` subflow module to open raw data files. It includes options such as saving raw files, using raw data in offline mode, and utilizing offline data.

- **echoflow_compute_TS**: This stage employs the `compute_TS` subflow module to compute target strength. It includes an option to use offline data.

**Note**: For a more comprehensive understanding of each option and its functionality, you can refer to the [Pipeline documentation](https://github.com/OSOceanAcoustics/echoflow/blob/dev/docs/configuration/pipeline.md).

Keep in mind that in this example, we'll be setting up a local Dask Cluster with 5 workers for parallel processing. This configuration will enable us to efficiently process our data for target strength analysis. To turn it off, toggle `use_local_dask` to false.

Feel free to explore and modify the configuration to understand better.

## Step 1.2
### Datastore Configuration: Organizing Data for Processing

In this section, we'll delve into the configuration that defines how the data will be organized and managed for processing. This configuration is provided in YAML format and plays a crucial role in structuring data inputs and outputs.

Here's the detailed breakdown of the configuration:

```yaml
name: Bell_M._Shimada-SH1707-EK60
sonar_model: EK60 
raw_regex: (.*)-?D(?P<date>\w{1,8})-T(?P<time>\w{1,6}) 
args: 
  urlpath: s3://ncei-wcsd-archive/data/raw/{{ ship_name }}/{{ survey_name }}/{{ sonar_model }}/*.raw
  parameters:
    ship_name: Bell_M._Shimada
    survey_name: SH1707
    sonar_model: EK60
  storage_options:
    anon: true
  transect:
    file: ./EK60_SH1707_Shimada.txt
  default_transect_num: 2017
  json_export: true 
output: 
  urlpath: ./echoflow-output
  retention: false
  overwrite: true
```

Let's delve into the individual components of the configuration presented here:

- **name**: Specifies a descriptive name for the configuration, aiding in identifying its purpose.

- **sonar_model**: Indicates the type of sonar being utilized, which in this case is "EK60".

- **raw_regex**: Defines a regular expression pattern for extracting date and time information from raw data file names.

- **args**: This section provides crucial arguments for structuring data inputs:

  - **urlpath**: Defines the URL pattern to access the raw data files stored on a remote server. The placeholders `{{ ship_name }}`, `{{ survey_name }}`, and `{{ sonar_model }}` are dynamically replaced with the specified values.

  - **storage_options**: Sets storage options, such as anonymous access (`anon: true`), for retrieving the data.

  - **transect**: Specifies a file (`EK60_SH1707_Shimada.txt`) containing the list of files to process, along with default transect information.

  - **json_export**: Enables JSON metadata export.

- **output**: This section configures the output settings for processed data:

  - **urlpath**: Determines the output directory (`./echoflow-output`) where the processed data will be stored.

  - **retention**: Disables data retention, indicating that only Target Strength data will be stored in this case.

  - **overwrite**: Allows data overwriting if the data already exists.

**Note**: 
- For a more comprehensive understanding of each option and its functionality, you can refer to the [Datast documentation](https://github.com/OSOceanAcoustics/echoflow/blob/dev/docs/configuration/datastore.md).
- The pipeline will store Target Strength output under `./echoflow-output`. As the retention is set to false, only Target Strength files will be stored. To specify files for processing, create a list of file names and store it in `EK60_SH1707_Shimada.txt`, which should be placed under the transect directory.

This configuration facilitates efficient data organization and management for the processing pipeline. Feel free to tailor it to your specific data and processing requirements.


## Step 2: Getting Data
Next, we'll use the glob_url function to retrieve a list of URLs matching a specific pattern. In this case, we're targeting raw EK60 data files from the SH1707 survey.

In [2]:
all_files = glob_url("s3://ncei-wcsd-archive/data/raw/Bell_M._Shimada/SH1707/EK60/*.raw", {'anon':True})

## Step 3: Preparing Files
We'll now extract the file names from the URLs and create a file listing for the transect. This will help us organize and work with the data effectively.

In [3]:
files = []
for file in all_files:
    f = file.split(".r")[0]
    files.append(f.split("/")[-1])

transect = open('EK60_SH1707_Shimada.txt','w')
i = 0
for f in files:
    if i == 10:
        break
    transect.write(f+".raw\n")
    i = i + 1
transect.close()

## Step 4: Processing with echoflow
Now, we're ready to kick off the data processing using echoflow. We'll provide the dataset and pipeline configurations, along with additional options.

In [4]:
options = {"storage_options_override": False}
data  = echoflow_start(dataset_config=dataset_config, pipeline_config=pipeline_config, options=options)


Checking Configuration

Configuration Check Completed
Checking Connection to Prefect Server

Starting the Pipeline


Dataset Configuration Loaded For This Run
--------------------------------------------------
{'name': 'Bell_M._Shimada-SH1707-EK60', 'sonar_model': 'EK60', 'raw_regex': '(.*)-?D(?P<date>\\w{1,8})-T(?P<time>\\w{1,6})', 'args': {'urlpath': 's3://ncei-wcsd-archive/data/raw/{{ ship_name }}/{{ survey_name }}/{{ sonar_model }}/*.raw', 'parameters': {'ship_name': 'Bell_M._Shimada', 'survey_name': 'SH1707', 'sonar_model': 'EK60'}, 'storage_options': {'anon': True}, 'transect': {'file': './EK60_SH1707_Shimada.txt'}, 'default_transect_num': 2017, 'json_export': True}, 'output': {'urlpath': './echoflow-output', 'retention': True, 'overwrite': True}}
Pipeline Configuration Loaded For This Run
--------------------------------------------------
{'active_recipe': 'target_strength', 'use_local_dask': False, 'n_workers': 5, 'pipeline': [{'recipe_name': 'target_strength', 'stages': [{'na

20:16:16.235 | ERROR   | Task run 'club_raw_files-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 1760, in orchestrate_task_run
    result = await call.aresult()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 293, in aresult
    return await asyncio.wrap_future(self.future)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\tasks.py", line 304, in __wakeup
    future.result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 318, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\echoflow\aspects\echoflow_aspect.py", line 97, in wrapper
    raise e
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\echoflow\aspects\echoflow_aspect.py", line 90, in wrapper
    result = func(*args, **kwargs)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\echoflow\utils\config_utils.py", line 440, in club_raw_files
    raise ValueError("Must have raw_dicts or raw_json_path present.")
ValueError: Must have raw_dicts or raw_json_path present.

20:16:16.303 | ERROR   | Task run 'club_raw_files-0' - Finished in state Failed('Task run encountered an exception ValueError: Must have raw_dicts or raw_json_path present.')

20:16:16.306 | ERROR   | Flow run 'incredible-capuchin' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 841, in orchestrate_flow_run
    result = await flow_call.aresult()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 293, in aresult
    return await asyncio.wrap_future(self.future)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\tasks.py", line 304, in __wakeup
    future.result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 318, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\echoflow\aspects\echoflow_aspect.py", line 97, in wrapper
    raise e
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\echoflow\aspects\echoflow_aspect.py", line 90, in wrapper
    result = func(*args, **kwargs)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\echoflow\stages\subflows\initialization_flow.py", line 74, in init_flow
    data = club_raw_files(
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\tasks.py", line 549, in __call__
    return enter_task_run_engine(
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 1156, in enter_task_run_engine
    return from_sync.wait_for_call_in_loop_thread(begin_run)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\api.py", line 243, in wait_for_call_in_loop_thread
    return call.result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 284, in result
    return self.future.result(timeout=timeout)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 168, in result
    return self.__get_result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\concurrent\futures\_base.py", line 403, in __get_result
    raise self._exception
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 355, in _run_async
    result = await coro
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 1324, in get_task_call_return_value
    return await future._result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\futures.py", line 237, in _result
    return await final_state.result(raise_on_failure=raise_on_failure, fetch=True)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\states.py", line 91, in _get_state_result
    raise await get_state_exception(state)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 1760, in orchestrate_task_run
    result = await call.aresult()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 293, in aresult
    return await asyncio.wrap_future(self.future)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\tasks.py", line 304, in __wakeup
    future.result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 31

20:16:16.384 | ERROR   | Flow run 'incredible-capuchin' - Finished in state Failed('Flow run encountered an exception. ValueError: Must have raw_dicts or raw_json_path present.')

20:16:16.386 | ERROR   | Flow run 'peach-warthog' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 841, in orchestrate_flow_run
    result = await flow_call.aresult()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 293, in aresult
    return await asyncio.wrap_future(self.future)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\tasks.py", line 304, in __wakeup
    future.result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 318, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\echoflow\stages\echoflow_trigger.py", line 170, in echoflow_trigger
    return init_flow(config=dataset, pipeline=pipeline, json_data_path=json_data_path)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\flows.py", line 1097, in __call__
    return enter_flow_run_engine_from_flow_call(
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 283, in enter_flow_run_engine_from_flow_call
    retval = from_sync.wait_for_call_in_loop_thread(
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\api.py", line 243, in wait_for_call_in_loop_thread
    return call.result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 284, in result
    return self.future.result(timeout=timeout)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 168, in result
    return self.__get_result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\concurrent\futures\_base.py", line 403, in __get_result
    raise self._exception
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 355, in _run_async
    result = await coro
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\client\utilities.py", line 51, in with_injected_client
    return await fn(*args, **kwargs)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 725, in create_and_begin_subflow_run
    return await terminal_state.result(fetch=True)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\states.py", line 91, in _get_state_result
    raise await get_state_exception(state)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\engine.py", line 841, in orchestrate_flow_run
    result = await flow_call.aresult()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 293, in aresult
    return await asyncio.wrap_future(self.future)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 285, in __await__
    yield self  # This tells Task to wait for completion.
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\tasks.py", line 304, in __wakeup
    future.result()
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\asyncio\futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\prefect\_internal\concurrency\calls.py", line 318, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
  File "c:\Users\soham\anaconda3\envs\echoflow\lib\site-packages\echoflow\aspects\echoflow_aspect.py", line 97, in wrapper
    raise e
  File "c:\Users\soham\an

20:16:16.432 | ERROR   | Flow run 'peach-warthog' - Finished in state Failed('Flow run encountered an exception. ValueError: Must have raw_dicts or raw_json_path present.')

ValueError: Must have raw_dicts or raw_json_path present.

## Step 5: Results
Finally, let's take a look at the first entry from the processed data.

In [ ]:
data[0][0]

**Congratulations!** You've successfully processed EK60 data using echoflow. This notebook provides a simplified overview, and you can explore the capabilities of echoflow for more advanced processing tasks.

Feel free to modify the parameters, paths, and configurations as needed to adapt to your data and requirements.